## 5. Preparación de Datos

[Playlist de Ciencia de Datos en castellano](https://www.youtube.com/watch?v=tBfGYKITno8&list=PLLBUgWXdTBDg1Qgmwt4jKtVn9BWh5-zgy)
[![Ciencia de Datos en Python](https://img1.wsimg.com/isteam/ip/aab852a2-7b1f-49c0-92af-9206f2ec6a75/6-0001.png/:/rs=w:1160,h:653)](https://www.youtube.com/watch?v=tBfGYKITno8&list=PLLBUgWXdTBDg1Qgmwt4jKtVn9BWh5-zgy "Python Data Science")

Gran parte del trabajo en Ciencia de Datos y Aprendizaje Automático (Machine Learning) consiste en obtener datos limpios y en la forma correcta. Esto puede incluir limpieza de datos para eliminar valores atípicos o mala información, escalado para algoritmos de aprendizaje automático o machine learning, división en grupos de entrenamiento y prueba, y enumeración de datos tipo "string". Todo esto debe suceder antes de la regresión, clasificación u otro entrenamiento aplicado al modelo. Afortunadamente, existen funciones que nos ayudan a automatizar la preparación de los datos.

![idea](https://apmonitor.com/che263/uploads/Begin_Python/idea.png)

### Generar Datos de Muestra

Ejecuta la siguiente celda para generar datos de muestra dañados con NaN (not a number) y valores atípicos que son puntos de datos erróneos que están muy por fuera de la tendencia esperada.

In [ ]:
import numpy as np
import pandas as pd
np.random.seed(1)
n = 100
tt = np.linspace(0,n-1,n)
x = np.random.rand(n)+10+np.sqrt(tt)
y = np.random.normal(10,x*0.01,n)
x[1] = np.nan; y[2] = np.nan  # 2 NaN (not a number)
for i in range(3):            # añade 3 valores atípicos (datos malos)
    ri = np.random.randint(0,n)
    x[ri] += np.random.rand()*100
data = pd.DataFrame(np.vstack((tt,x,y)).T,\
                    columns=['time','x','y'])
data.head()

![analyze](https://apmonitor.com/che263/uploads/Begin_Python/analyze.png)

### Visualización de los Datos

Los valores atípicos se muestran en un gráfico semi-log. Los valores `NaN` no se muestran en la gráfica y son puntos faltantes.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.semilogy(tt,x,'r.',label='x')
plt.semilogy(tt,y,'b.',label='y')
plt.legend(); plt.xlabel('tiempo')
plt.text(50,60,'Valores atípicos')
plt.show()

![idea](https://apmonitor.com/che263/uploads/Begin_Python/idea.png)

### Eliminar Valores Atípicos y Datos Erróneos

Los valores NaN se eliminan con `numpy` identificando filas `ix` que contienen `NaN`. A continuación, las filas se eliminan con `z=z[~iz]` donde `~` Es un operador `not`.

In [ ]:
z = np.array([[      1,      2],
              [ np.nan,      3],
              [      4, np.nan],
              [      5,      6]])
iz = np.any(np.isnan(z), axis=1)
print(~iz)
z = z[~iz]
print(z)

El método `dropna` es un comando que quita filas `NaN` en un `pandas` `DataFrame`. La fila 1 y 2 son removidas.

In [ ]:
# eliminar cualquier fila con valores erróneos (NaN)
data = data.dropna()
data.head()

Existen varias técnicas gráficas para detectar valores atípicos. Un diagrama de caja o histograma muestra los 3 puntos periféricos.

In [ ]:
plt.boxplot(data['x'])
plt.show()

Una prueba de Grubbs u [otra medida estadística](https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba) puede detectar valores atípicos. La prueba de Grubbs asume datos univariados, normalmente distribuidos y está destinada a detectar solo un valor atípico. En la práctica, muchos valores atípicos se eliminan al remover puntos que violan un límite de cambio, o límites superior/inferior. El enunciado `data[data['x']<30]` mantiene las filas donde x es menor a 30.

In [ ]:
data = data[data['x']<30]
plt.boxplot(data['x'])
plt.show()

![expert](https://apmonitor.com/che263/uploads/Begin_Python/expert.png)

### Actividad con Tiempo

Sin mirar tu reloj, ejecuta la siguiente celda para registrar intervalos de 1 segundo durante 10 segundos. Cuando ejecutes la celda, presiona `Enter` cada vez que creas que ha pasado 1 segundo. Después de recopilar los datos, utiliza un diagrama de caja para identificar cualquier punto de datos en `tsec` que son valores atípicos.

In [ ]:
import time
from IPython.display import clear_output
tsec = []
input('Presiona "Enter" para grabar intervalos de 1 segundo'); t = time.time()
for i in range(10):
    clear_output(); input('Presiona "Enter": ' + str(i+1))
    tsec.append(time.time()-t); t = time.time()
clear_output(); print('Completo. Agrega un diagrama de caja para identificar valores atípicos')

In [ ]:
# Agregar un diagrama de caja para identificar valores atípicos

![idea](https://apmonitor.com/che263/uploads/Begin_Python/idea.png)

### Escala de Datos

La librería `sklearn` tiene un módulo de pre-procesamiento (`preprocessing`) para implementar métodos de escalado estándar. El `StandardScalar` se muestra a continuación. Cada columna se normaliza a una media cero y una desviación estándar de uno. Los métodos de escalado comunes `fit_transform(X)` para ajuste y `transform(X)` transformación basado en otro ajuste, y `inverse_transform(Xs)` para volver a escalar a la representación original.

In [ ]:
from sklearn.preprocessing import StandardScaler
s = StandardScaler()
ds = s.fit_transform(data)
print(ds[0:5]) # imprime 5 filas

El valor `ds` se devuelve como un array `numpy` así que tenemos que convertirlo de nuevo a `pandas` `DataFrame`, reutilizando los nombres de columna `data`.

In [ ]:
ds = pd.DataFrame(ds,columns=data.columns)
ds.head()

![idea](https://apmonitor.com/che263/uploads/Begin_Python/idea.png)

### Dividir Datos

Los datos se dividen en grupos de entrenamiento y prueba para separar una fracción de las filas para evaluar modelos de clasificación o regresión. Una división típica es 80% para entrenamiento y 20% para pruebas, aunque el rango depende de la cantidad de datos disponibles y del objetivo del estudio.

In [ ]:
divide = int(len(ds)*0.8)
train = ds[0:divide]
test = ds[divide:]
print(len(train),len(test))

El `train_test_split` es una función de `sklearn` que tiene el propósito específico de dividir los datos en grupos de entrenamiento y prueba. Existen opciones como `shuffle=True` para aleatorizar la selección en cada conjunto. 

In [ ]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(ds, test_size=0.2, shuffle=True)
print(len(train),len(test))

### Actividad con el TCLab

![expert](https://apmonitor.com/che263/uploads/Begin_Python/expert.png)

### Datos con Valores Erróneos y Atípicos

Genera un nuevo archivo de datos con algunos datos incorrectos insertados aleatoriamente (3 minutos) o lee el archivo de datos de [un link en la web](https://apmonitor.com/do/uploads/Main/tclab_bad_data.txt) con el siguiente código.

In [ ]:
import tclab, time, csv
import numpy as np

try:
    with tclab.TCLab() as lab:
        with open('05-tclab.csv',mode='w',newline='') as f:
            cw = csv.writer(f)
            cw.writerow(['Time','Q1','Q2','T1','T2'])
            print('t Q1 Q2 T1    T2')
            for t in range(180):
                T1 = lab.T1; T2 = lab.T2
                # insertar valores malos
                bad = np.random.randint(0,30)
                T1=np.nan if bad==10 else T1
                T2=np.nan if bad==15 else T2
                # insertar número aleatorio (potencial valor atípico)
                outlier = np.random.randint(-40,150)
                T1=outlier if bad==20 else T1
                T2=outlier if bad==25 else T2
                # cambiar el calentador
                if t%30==0:
                    Q1 = np.random.randint(0,81)
                    Q2 = np.random.randint(0,81)
                    lab.Q1(Q1); lab.Q2(Q2)
                cw.writerow([t,Q1,Q2,T1,T2])
                if t%10==0:
                    print(t,Q1,Q2,T1,T2)
                time.sleep(1)
            data5=pd.read_csv('05-tclab.csv')
except:
    print('Conectar el TCLab para generar nuevos datos')
    print('Importar datos de un repositorio en línea')
    url = 'http://apmonitor.com/do/uploads/Main/tclab_bad_data.txt'
    data5=pd.read_csv(url)

### Limpiar, Escalar y Dividir Datos

Después de generar e importar `data5`, eliminar filas con valores `NaN` o valores atípicos en las columnas `T1` o `T2`. Escala los datos con `StandardScalar` en `scikit`. Divide los datos en grupos de entrenamiento (80%) y prueba (20%).  